# Tidsserier med Spark

I denne notebooken vises noen enkle eksempler på hvordan du kan jobbe med tidsserier i PySpark. Denne notebooken er kjørt i prodmiljøet på **Dapla** med `Pyspark (local)`-kernel.

In [ ]:
# Importer biblioteker
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, explode, expr, sequence
from pyspark.sql.types import DateType, DoubleType, StructField, StructType

In [ ]:
# Initialize a SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

In [ ]:
# Generate a sequence of dates
dates_df = spark.range(1).select(
    explode(
        sequence(
            start=expr("date '2000-01-01'"),
            stop=expr("date '2023-08-01'"),
            step=expr("interval 1 month"),
        )
    ).alias("Date")
)
dates_df.show()

In [ ]:
# Genererer random walk data
schema = StructType(
    [StructField(f"serie{i:02d}", DoubleType(), True) for i in range(100)]
)

data = [
    tuple((10 + np.random.normal(0, 1, 100)).cumsum().tolist())
    for _ in range(284)  # 284 months from 2000-01 to 2023-08
]

data_df = spark.createDataFrame(data, schema=schema)

data_df.select("serie00", "serie01").show()

In [ ]:
# Legger til row index til DataFrame før join med dates_df
data_df = data_df.withColumn("row_index", expr("monotonically_increasing_id()"))

# Joiner de to datasettene
df = (
    dates_df.withColumn("row_index", expr("monotonically_increasing_id()"))
    .join(data_df, "row_index")
    .drop("row_index")
)

# Legger til år, kvartal og mnd
df = df.withColumn("Year", date_format(df.Date, "yyyy"))
df = df.withColumn("Quarter", expr("quarter(Date)"))
df = df.withColumn("Month", date_format(df.Date, "MM"))

df.select("Date",  "Year", "Quarter", "Month", "serie00", "serie01").show()